<a href="https://colab.research.google.com/github/jsmazorra/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/DS-Unit-4-Sprint-2-Neural-Networks/Johan_Mazorra_LS_DS13_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [27]:
# Let's import the necessary libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

# Let's import the telecom customer churn dataset.
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.shape

(7043, 21)

In [5]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [6]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,6234-RAAPL,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,20.2,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [0]:
# Let's clean up the data.

encoder = OneHotEncoder()
label_encoder = LabelEncoder()

binary = ['gender', 'Partner', 'Dependents', 'PhoneService', 
          'PaperlessBilling', 'Churn']
multi = ['MultipleLines', 'InternetService', 'OnlineSecurity',
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
        'StreamingMovies', 'Contract', 'PaymentMethod', 'TotalCharges']


In [0]:
for col in binary:
    df[col] = encoder.fit_transform(df[[col]]).todense()
    df[col] = df[col].astype(int)

In [9]:
for col in multi:
    df[col] = label_encoder.fit_transform(df[[col]])
    df[col] = df[col].astype(int)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,0,1,1,1,1,0,0,2,0,0,0,0,0,0,2,29.85,2505,1
1,5575-GNVDE,0,0,1,1,34,0,0,0,2,0,2,0,0,0,1,1,3,56.95,1466,1
2,3668-QPYBK,0,0,1,1,2,0,0,0,2,2,0,0,0,0,0,0,3,53.85,157,0
3,7795-CFOCW,0,0,1,1,45,1,1,0,2,0,2,2,0,0,1,1,0,42.30,1400,1
4,9237-HQITU,1,0,1,1,2,0,0,1,0,0,0,0,0,0,0,0,2,70.70,925,0


In [11]:
df.shape

(7043, 21)

In [12]:
# Let's take out churn for testing and remove useless columns.

X = df.drop(columns=['Churn', 'customerID', 'TotalCharges'])
y = df['Churn']

X.shape, y.shape

((7043, 18), (7043,))

In [0]:
# Scale the data.
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
# Now let's fit a model.
inputs = X.shape[1]
epochs = 20
batch_size = 42

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=inputs))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                608       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X, y, 
          validation_split=0.25, 
          epochs=epochs, 
          batch_size=batch_size)

Epoch 1/20
126/126 [==============================] - 1s 4ms/step - loss: 1.6689 - accuracy: 0.6304 - val_loss: 1.3936 - val_accuracy: 0.7217
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.9326 - accuracy: 0.7270 - val_loss: 0.9814 - val_accuracy: 0.7263
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.8105 - accuracy: 0.7255 - val_loss: 0.9107 - val_accuracy: 0.7291
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.7332 - accuracy: 0.7385 - val_loss: 0.9314 - val_accuracy: 0.7518
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.6975 - accuracy: 0.7397 - val_loss: 0.7972 - val_accuracy: 0.7496
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.6539 - accuracy: 0.7399 - val_loss: 0.7455 - val_accuracy: 0.7467
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.6316 - accuracy: 0.7491 - val_loss: 0.7369 - val_accuracy: 0.7518
Epoch 

In [16]:
model.evaluate(X, y)

221/221 [==============================] - 0s 2ms/step - loss: 0.5454 - accuracy: 0.7809


[0.5453972220420837, 0.780917227268219]

In [33]:
# Let's create a new model.
def create_model():
    model = Sequential()
    model.add(Dense(1, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [8, 16, 32, 64, 128],
              'epochs': [20]}

# Then let's create the Grid Search.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8032087802886962 using {'batch_size': 16, 'epochs': 20}
Means: 0.8015036225318909, Stdev: 0.010367324863276881 with: {'batch_size': 8, 'epochs': 20}
Means: 0.8032087802886962, Stdev: 0.006976752046419831 with: {'batch_size': 16, 'epochs': 20}
Means: 0.8022144556045532, Stdev: 0.009282281273895073 with: {'batch_size': 32, 'epochs': 20}
Means: 0.7969602823257447, Stdev: 0.0059127203719791036 with: {'batch_size': 64, 'epochs': 20}
Means: 0.7792154908180237, Stdev: 0.009206145316134767 with: {'batch_size': 128, 'epochs': 20}


In [40]:
# Let's try a new model for better results.
def create_model(activation_function="relu"):
    opt = Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(1, activation=activation_function))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [8],
              'epochs': [16],
              "activation_function": ["relu", "selu", "elu", "exponential","linear","swish"]}

# Then let's create the Grid Search.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7983828663825989 using {'activation_function': 'selu', 'batch_size': 8, 'epochs': 16}
Means: 0.7696889638900757, Stdev: 0.03379139938493232 with: {'activation_function': 'relu', 'batch_size': 8, 'epochs': 16}
Means: 0.7983828663825989, Stdev: 0.005000127295661133 with: {'activation_function': 'selu', 'batch_size': 8, 'epochs': 16}
Means: 0.7956833958625793, Stdev: 0.004556253043808484 with: {'activation_function': 'elu', 'batch_size': 8, 'epochs': 16}
Means: 0.7681428074836731, Stdev: 0.026930187150625128 with: {'activation_function': 'exponential', 'batch_size': 8, 'epochs': 16}
Means: 0.7966786026954651, Stdev: 0.0054510194810143825 with: {'activation_function': 'linear', 'batch_size': 8, 'epochs': 16}
Means: 0.7871684312820435, Stdev: 0.025505599369347355 with: {'activation_function': 'swish', 'batch_size': 8, 'epochs': 16}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?